# Visualizing Population and Unemployment Data

In [ ]:
!lscpu

In [ ]:
!free -g

In [ ]:
# !sudo pip3 install cython
# !sudo pip3 install cartopy
# !sudo pip3 install geoplot
# !sudo pip3 install descartes
# !sudo pip3 install pysal
# !sudo pip3 install geopandas

In [ ]:
# !sudo -H pip3 install -U pip cython cartopy geoplot descartes pysal geopandas pandas numpy requests

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import requests # to connect to the Census API
import ast # for parsing the Census API response

import pandas as pd
import numpy as np

import geopandas as gpd
import geoplot
import geoplot.crs as gcrs

In [ ]:
# To plot the choropleths, we need the shapefiles for the areas. Below we use 
# shapefiles that come from the US Census data
# 
# More files at https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html
# 
# Check also http://geojson.xyz/ for more shapefiles
#
shapefiles_zipcodes = "http://www2.census.gov/geo/tiger/GENZ2017/shp/cb_2017_us_zcta510_500k.zip"
shapefiles_counties = "http://www2.census.gov/geo/tiger/GENZ2017/shp/cb_2017_us_county_500k.zip"
shapefiles_states   = "http://www2.census.gov/geo/tiger/GENZ2017/shp/cb_2017_us_state_500k.zip"

## Load GeoDataframe for US States and plot a map

In [ ]:
df_states = gpd.read_file(shapefiles_states)

In [ ]:
# df_states

In [ ]:
# Dropping non-continental states
df_states = df_states[ df_states.NAME!='Alaska' ] 
df_states = df_states[ df_states.NAME!='Hawaii' ] 
df_states = df_states[ df_states.NAME!='Puerto Rico' ] 
df_states = df_states[ df_states.NAME!='Guam' ] 
df_states = df_states[ df_states.NAME!='Commonwealth of the Northern Mariana Islands' ] 
df_states = df_states[ df_states.NAME!='American Samoa' ] 
df_states = df_states[ df_states.NAME!='United States Virgin Islands' ] 

In [ ]:
# Should be 49. The 48 continental, plus DC
assert( len(df_states) == 49 )

In [ ]:
df_states.plot(
    figsize=(10,10), 
    linewidth=0.2, 
    color='white', 
    edgecolor='black'
)

## Load US Counties Datafame and plot a map

In [ ]:
df_counties = gpd.read_file(shapefiles_counties)

In [ ]:
# df_counties

In [ ]:
len(df_counties)

In [ ]:
# There should be 3233 counties looaded
assert( len(df_counties) == 3233 )

In [ ]:
# Keep only counties in the lower 48 states + DC
keep_county = df_counties.STATEFP.isin(df_states.STATEFP.values)
df_counties = df_counties[ keep_county ]

In [ ]:
# There should be 3108 counties remaining
assert(len(df_counties) == 3108)

In [ ]:
df_counties.plot(
    figsize=(15,15), 
    linewidth=0.2,  
    color='white', 
    edgecolor='black'
)

## Getting Census Data

You need to get an API Key from http://api.census.gov/data/key_signup.html.

In [ ]:
class Census:
    def __init__(self, key):
        self.key = key

    def get(self, fields, geo, year=2010, dataset='sf1'):
        fields = [','.join(fields)]
        template_url = 'https://api.census.gov/data/{year}/{dataset}?key={key}&get='
        base_url = template_url.format(year=str(year), dataset=dataset, key=self.key)
        query = fields
        for item in geo:
            query.append(item)
        add_url = '&'.join(query)
        url = base_url + add_url
        response = requests.get(url)
        return ast.literal_eval(response.text) 

api_key = '627d4107b57d4576f2120d2b87b59c7440e5d2af'
census = Census(api_key)

## Plot a Choropleth with Population of US States

In [ ]:
# Fetch state population data from US Census
census_response = census.get(['P0010001'], ['for=state:*'])
# Manipulate the result from the US Census API and convert the result to a dataframe
df_state_population = pd.DataFrame(census_response[1:], columns = ['Population', 'STATEFP'])
df_state_population['Population'] = pd.to_numeric(df_state_population['Population'])
df_state_population['LogPopulation'] = np.log10(df_state_population['Population'])
# df_state_population

In [ ]:
# ax = df_state_population.LogPopulation.hist()
# df_state_population.LogPopulation.plot.kde(secondary_y=True, ax = ax)

In [ ]:
# Augment the df_states geodataframe with population information
states_choropleth = pd.merge(df_states, df_state_population, on='STATEFP')

states_choropleth.plot(
    figsize=(25,25), 
    column='LogPopulation', 
    cmap='Blues',  # select color scheme from http://matplotlib.org/users/colormaps.html
    linewidth=0.1, 
    edgecolor='black'
)

## Plot a Choropleth with Population of US Counties

In [ ]:
# Query the US Census API for the population of each county
census_response = census.get(['P0010001'], ['in=state:*', 'for=county:*'])
# Manipulate the API response and put the results in a dataframe
df_county_population = pd.DataFrame(census_response[1:], columns = ['Population', 'STATEFP', 'COUNTYFP'])
df_county_population['Population'] = pd.to_numeric(df_county_population['Population'])
df_county_population['LogPopulation'] = np.log10(df_county_population['Population'])
# df_county_population

In [ ]:
df_county_population.head(5)

In [ ]:
df_county_population.LogPopulation.plot.kde()
df_county_population.LogPopulation.hist(bins=40, density=True)

In [ ]:
# Merge the GeoDataFrame df_counties with the population data
counties_choropleth = pd.merge(df_counties, df_county_population, on=['STATEFP', 'COUNTYFP'])

counties_choropleth.plot(
    figsize=(15,15), 
    column='LogPopulation', 
    cmap='Blues', # http://matplotlib.org/users/colormaps.html
    # scheme='Quantiles', # alternatives are 'Quantiles', Equal_Interval', and 'Fisher_Jenks'; Quantiles requires PySAL
    linewidth=0.1, 
    edgecolor='black')

## Extras

In [ ]:
# Plot the state borders (with darker, thicker lines) on top of the counties

ax = counties_choropleth.plot(
    figsize=(15,15), 
    column='LogPopulation', 
    cmap='Blues', # http://matplotlib.org/users/colormaps.html
    # scheme='Quantiles', # alternatives are 'Quantiles', Equal_Interval', and 'Fisher_Jenks'; Quantiles requires PySAL
    linewidth=0.1, 
    edgecolor='black')

df_states.plot(
    figsize=(15,15), 
    linewidth=1, # thicker line
    facecolor='none', # no color for fill
    edgecolor='#333366', # color for the state borders
    ax = ax # plot it on top of the counties plot
)


In [ ]:
# Changing the projection to Orthographic
geoplot.choropleth(
    counties_choropleth,
    hue='LogPopulation',
    categorical=False, # LogPopulation is continuous not categorical
    k=10, # split LogPopulation into 10 bins for coloring
    scheme='equal_interval', # Each bin has equal range for LogPopulation
    cmap='Blues', # try Spectral_r, Spectral, or others from https://matplotlib.org/tutorials/colors/colormaps.html
    linewidth=0.3,
    projection=gcrs.Orthographic(),
    figsize=(15, 15)
).gridlines() # plot gridlines

In [ ]:
# Add state borders in the geoplot choropleth plot

ax = geoplot.polyplot(
    states_choropleth, 
    projection=gcrs.Orthographic(),
    figsize=(15, 15),
    linewidth = 0.5,
    zorder = 2
)

geoplot.choropleth(
    counties_choropleth,
    hue='LogPopulation',
    categorical=False, # LogPopulation is continuous not categorical
    k=10, # split LogPopulation into 40 bins for coloring
    scheme='equal_interval', # Each bin has equal range for LogPopulation
    cmap='Blues', # try Spectral_r, Spectral, or others from https://matplotlib.org/tutorials/colors/colormaps.html
    linewidth=0.3,
    projection=gcrs.Orthographic(),
    figsize=(15, 15),
    ax = ax,
    zorder = 0
).gridlines(zorder=1) # plot gridlines



## Unemployment Data

https://www.bls.gov/lau/
    
 * Labor force data by county, not seasonally adjusted, latest 14 months https://www.bls.gov/web/metro/laucntycur14.txt
 
 * Labor force data by county, 2017 annual averages https://www.bls.gov/lau/laucnty17.txt 

In [ ]:
url = "https://www.bls.gov/web/metro/laucntycur14.txt"
udf = pd.read_csv(
    url,
    skiprows=6,  # skip the first six lines; 
    header=None,  # we will supply the headers ourselves
    skipfooter=6,  # the last six lines are notes; skip them
    engine='python',  # we need this to use the skipfooter option
    delimiter="|",  # use | as the column delimeter
    # skipinitialspace=True,  # ignore the space characters after the delimeter |
    # thousands=",",  # specify that numbers use , for thousand separator
)

In [ ]:
udf.head(5)

In [ ]:
udf.dtypes

In [ ]:
udf.columns = ["LAUS", "STATEFP", "COUNTYFP", "County_Name", "Period", 
               "Labor_Force", "Employed", "Unemployed", "Rate"]

In [ ]:
udf.head(5)

In [ ]:
udf.dtypes

In [ ]:
# Convert STATEFP and COUNTYFP to strings and add leading zeros
# to allow for merging later on with the geodataframes
# Interestingly, the initial file contains the zeros, but Pandas recognizes
# the entries as numbers and converts to integers, so we are forced to convert back
udf.STATEFP = udf.STATEFP.apply(str).str.zfill(2)
udf.COUNTYFP = udf.COUNTYFP.apply(str).str.zfill(3)

In [ ]:
udf.head(5)

In [ ]:
# Using the thousands=","  option in read_csv allows Pandas to understand the 
# data type of these columns properly. If we do not use the option, the 
# code below can convert the values into recognizable numbers to be used by to_numeric
# 
# Need to replace the comma-separator in the numbers with an empty character
# to allow for proper conversion
#
udf.Labor_Force = pd.to_numeric(udf.Labor_Force.str.replace(',',''))
udf.Employed = pd.to_numeric(udf.Employed.str.replace(',',''))
udf.Unemployed = pd.to_numeric(udf.Unemployed.str.replace(',',''))

In [ ]:
# We now want to convert the "Period" to datetime
# Let's take a look at the unique values
udf.Period.drop_duplicates()

In [ ]:
# Converting "Period" to a proper date
# We will need to remove spaces 
# We will also need to remove the notes (p) and (y); we need to escape the parentheses
udf.Period = udf.Period.str.replace(' ','')
udf.Period = udf.Period.str.replace('\(p\)','')
udf.Period = udf.Period.str.replace('\(y\)','')
udf.Period.drop_duplicates()

In [ ]:
udf.Period = pd.to_datetime(udf.Period, format='%b-%y')

In [ ]:
udf.head(5)

In [ ]:
udf.dtypes

In [ ]:
unemployment_average = udf.pivot_table(
    index = ['STATEFP','COUNTYFP', 'County_Name'],
    values = ['Rate','Labor_Force'],
    aggfunc = 'mean'
)

unemployment_average = unemployment_average.reset_index()
unemployment_average.head(5)

In [ ]:
unemployment_average.Rate.describe()

In [ ]:
# County with the highest unemployment rate
unemployment_average [ unemployment_average.Rate == unemployment_average.Rate.max() ]

In [ ]:
# County with the lowest unemployment rate
unemployment_average [ unemployment_average.Rate == unemployment_average.Rate.min() ]

In [ ]:
# County with the median unemployment rate
unemployment_average [ unemployment_average.Rate == unemployment_average.Rate.median() ]

In [ ]:
unemployment_average.Rate.hist(bins=100,range=(0,20))

In [ ]:
unemployment_average.Rate.plot.kde(xlim=(0,20))

In [ ]:
ax = unemployment_average.Rate.hist(bins=100,range=(0,20), density=True, alpha=0.5)
ax = unemployment_average.Rate.plot.kde(xlim=(0,20), ax=ax, linewidth=3)
ax.set_xlabel("Unemployment Rate (%)")

In [ ]:
# Merge the GeoDataFrame df_counties with the unemployment data
# Note that this will drop the rates for counties in Alaska and Hawaii
# as this is an inner join, and we have dropped from df_counties Alaska and Hawaii
unemployment_choropleth = pd.merge(df_counties, unemployment_average, on=['STATEFP', 'COUNTYFP'])

In [ ]:
unemployment_choropleth.head(5)

In [ ]:
# Compare with https://www.bls.gov/web/metro/twmcort.pdf

ax = geoplot.polyplot(
    states_choropleth, 
    projection=gcrs.Orthographic(),
    figsize=(15, 15),
    linewidth = 0.5,
    zorder = 2
)

geoplot.choropleth(
    unemployment_choropleth,
    hue='Rate',
    categorical=False, # Unemployment rate is continuous not categorical
    k=10, # split Unemployment into 10 bins for coloring
    scheme='quantiles', # Use quantiles for creating the bins
    cmap='coolwarm', # We use a **divergent** colormap, to show places with higher than average and lower than average https://matplotlib.org/tutorials/colors/colormaps.html
    linewidth=0.3,
    projection=gcrs.Orthographic(),
    figsize=(15, 15),
    ax = ax,
    zorder = 0,
    legend = True,
    legend_kwargs={'loc': 'lower right'}
).gridlines(zorder=1) # plot gridlines on top of the counties (zorder=0), but behind state lines (zorder=2)



### Extra: Saving quantiles as a separate column

In [ ]:
num_quantiles = 10

In [ ]:
quantiles = pd.qcut(unemployment_choropleth.Rate, num_quantiles)

In [ ]:
# Let's see the ranges for the quantiles
quantiles.drop_duplicates().sort_values()

In [ ]:
# We use the labels=Fase to get a number (0,1,2,3...) instead of the range labels 
unemployment_choropleth["qRate"] = pd.qcut(unemployment_choropleth.Rate, num_quantiles, labels=False)
unemployment_choropleth["Rate_range"] = pd.qcut(unemployment_choropleth.Rate, num_quantiles)

In [ ]:
unemployment_choropleth.head(5)

### Mitigating map visualization bias: Area vs Population

In the example above, where we visualize unemployment rates per county, we introduce a bias: Areas with large area get more prominently displayed. We can try to mitigate this bias, by using cartograms, which scale each area based on a factor that we desire. 

* **County weight proportional to county area**: Scaling factor `constant` (the default).
* **County weight equal across all counties**: Scaling factor proportional to `1/area`
* **County weight proportional to county population**: Scaling factor proportional to `population/area`

So, for example, if we want each country to have an area proportional to its **population** instead of its **area**, we may want to use Density (population divided by area)

In [ ]:
# Density = Population / Area
unemployment_choropleth["Density"] = unemployment_choropleth.Labor_Force / unemployment_choropleth.ALAND

In [ ]:
# Normalize density to have a minimum value of 1... 
# Alternatively, we can do it to have a max value of 1 by dividing with max
unemployment_choropleth["Density"] = unemployment_choropleth.Density / unemployment_choropleth.Density.min()

In [ ]:
# Take the log, as densities are roughly log-normally distributed (as is population)
# We also normalize it by dividing with the max value (to make it 0...1 )
# Finally we add a multiplicative factor of 2
unemployment_choropleth["LogDensity"] = 2*np.log10(unemployment_choropleth.Density)/np.log10(unemployment_choropleth.Density.max())

In [ ]:
# Statistics on LogDensity values
unemployment_choropleth.LogDensity.describe()

In [ ]:
ax = unemployment_choropleth.LogDensity.hist(bins=50,range=(0,2), density=True, alpha=0.5)
ax = unemployment_choropleth.LogDensity.plot.kde(xlim=(0,2), ax=ax, linewidth=3)
ax.set_xlabel("County Population Density (log-scale)")

In [ ]:
unemployment_choropleth[ unemployment_choropleth.Density == unemployment_choropleth.Density.max()]

In [ ]:
density_min = unemployment_choropleth.LogDensity.min()
density_max = unemployment_choropleth.LogDensity.max()

In [ ]:
density_min

In [ ]:
density_max

In [ ]:
geoplot.cartogram(
    unemployment_choropleth,
    scale='LogDensity',
    limits=(density_min,density_max),
    trace=False,
    projection=gcrs.Orthographic(),
    scheme='quantiles',
    hue='Rate',
    k=10,
    cmap='coolwarm',
    figsize=(30, 30))

In [ ]:
# Instead of doing manipulations of the density column on the dataframe,
# we can instead write a scaling function
def log_scale(minval, maxval):
    def scalar(val):
        max_scaling = 2
        min_scaling = 0.0
        normed_log = np.log10(val) - np.log10(minval)
        max_log = np.log10(maxval) - np.log10(minval)
        factor = (max_scaling-min_scaling) * normed_log/max_log + min_scaling
        return factor
    return scalar

geoplot.cartogram(
    unemployment_choropleth,
    scale='Density',
    scale_func=log_scale,
    trace=False,
    projection=gcrs.Orthographic(),
    scheme='quantiles',
    hue='Rate',
    k=10,
    cmap='coolwarm',
    figsize=(30, 30))